### Redes con tensorflow
#### Por Francisco Serradilla

#### Tareas

* [x] Entrenar el perceptrón multicapa suministrado.
* [x] Ampliarlo para meter más capas, usar activación relu y evaluar con un conjunto de test.
* [x] Entrenar con algunos de los problemas suministrados.
* (Opcional) Probar otros optimizadores.
* [x] (Opcional) Añadir dropout. ¿Mejora la generalización con alguno de los problemas suministrados?

### Introducción a tensorflow

In [1]:
# Creación de grafos

print('Loading tensorflow...')
import tensorflow as tf
print('Loaded')

# Create a Constant op that produces a 1x2 matrix.  The op is
# added as a node to the default graph.
# The value returned by the constructor represents the output of the Constant op.
matrix1 = tf.constant([[3., 3.]])

# Create another Constant that produces a 2x1 matrix.
matrix2 = tf.constant([[2.],[2.]])

# Create a Matmul op that takes 'matrix1' and 'matrix2' as inputs.
# The returned value, 'product', represents the result of the matrix multiplication.
product = tf.matmul(matrix1, matrix2)

product

Loading tensorflow...
Loaded


<tf.Tensor 'MatMul:0' shape=(1, 1) dtype=float32>

In [5]:
# Ejecutar un grafo

# Launch the default graph.
sess = tf.Session()
# To run the matmul op we call the session 'run()' method, passing ‘product' which represents the output of the matmul op.  This indicates to the call that we want to get the output of the matmul op back.
# All inputs needed by the op are run automatically by the session.  They typically are run in parallel.
# The call 'run(product)' thus causes the execution of three ops in the graph: the two constants and matmul. The output of the op is returned in 'result' as a numpy `ndarray` object.
result = sess.run(product)
print(result)
# ==> [[ 12.]]

# Close the Session when we're done.
sess.close()

[[12.]]


In [21]:
# Variables

# Create a Variable, that will be initialized to the scalar value 0.
state = tf.Variable(0, name="counter")

# Create an Op to add one to `state`.
one = tf.constant(1)
new_value = tf.add(state, one)
update = tf.assign(state, new_value)

# Variables must be initialized by running an `init` Op after having launched the graph.  
init_op = tf.global_variables_initializer() # We first have to add the `init` Op to the graph

with tf.Session() as sess: # Launch the graph and run the ops
    sess.run(init_op) # Run the 'init' op
    print(sess.run(state)) # Print the initial value of 'state'
    for _ in range(3): # Run the op that updates 'state' and print 'state'.
        sess.run(update)
        print(sess.run(state))

0
1
2
3


In [6]:
# fetches

input1 = tf.constant(3.0)
input2 = tf.constant(2.0)
input3 = tf.constant(5.0)
intermed = tf.add(input2, input3)
mul = tf.multiply(input1, intermed)

with tf.Session() as sess:
    result = sess.run([mul, intermed]) # fetches -> lo que se quiere obtener del cálculo
    print(result)

[21.0, 7.0]


In [7]:
# placeholders

input1 = tf.placeholder(tf.float32)
input2 = tf.placeholder(tf.float32)
output = tf.multiply(input1, input2)

with tf.Session() as sess:
    print(sess.run(output, feed_dict={input1:[7.], input2:[2.]}))

[14.]


In [8]:
# Carga ejemplos

import numpy as np

ni = 2 # número de entradas
nh = 10 # número de neuronas en capa oculta
no = 3 # número de neuronas en capa de salida
capas = 5 #número de capas ocultas de la red
datos_cargados = "circulo"

# carga datos de entrenamiento
d = np.loadtxt("samples/circulo.txt")
inputs = d[:,:ni]

outputs = d[:,ni:]


In [11]:
# Creación de la red
# Define un perceptrón multicapa con 2 capas usando RMS como medida del error y back proagation como algoritmo de entrenamiento

import tensorflow as tf

# un placeholder es un punto de comunicación de datos entre nuestra app y la librería tensorflow
# creamos un placeholder para indicar las entradas a la red de todos los ejemplos
e = tf.placeholder(tf.float32, [None, ni]) # None indica que la primera dimensión no es fija (el número de ejemplos)

# un nuevo placeholder para indicar las salidas deseadas
d = tf.placeholder(tf.float32, [None, no])

# las variables son matrices que residen fuera de python y no son modificadas directamente por nuestra app
W1 = tf.Variable(tf.random_uniform([ni, nh], -0.1, 0.1, dtype=tf.float32))
b1 = tf.Variable(tf.random_uniform([nh], -0.1, 0.1, dtype=tf.float32))
W2 = tf.Variable(tf.random_uniform([nh, no], -0.1, 0.1, dtype=tf.float32))
b2 = tf.Variable(tf.random_uniform([no], -0.1, 0.1, dtype=tf.float32))

# aquí definimos la propagación de la red
s1 = tf.nn.sigmoid(tf.add(tf.matmul(e, W1), b1))
s = tf.nn.sigmoid(tf.add(tf.matmul(s1, W2), b2))

print("\ndatos cargados:", datos_cargados,"\n")
print("s1")
print(s1)


datos cargados:
circulo

s1
Tensor("Sigmoid_4:0", shape=(?, 10), dtype=float32)


### Entrenar el perceptrón multicapa suministrado

In [14]:
# Entrenamiento

# definimos la función a minimizar (error cuadrático medio)
vRMS = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(d,s)), reduction_indices=0))
RMS = tf.reduce_mean(vRMS)

# definimos el algoritmo de aprendizaje a utilizar (descenso del gradiente sobre la función de coste)
train_step = tf.train.RMSPropOptimizer(0.01).minimize(RMS)

# definimos qué es un acierto  
correct_prediction = tf.equal(tf.argmax(s,1), tf.argmax(d,1))

# definimos la precisión como el porcentaje de aciertos
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# definimos la inicialización de las variables internas
init = tf.global_variables_initializer()

# iniciamos sesión con tensorflow y ejecutamos la inicialización
sess = tf.Session()
sess.run(init)

print("\nEntrenamiento para datos",datos_cargados, "\n")

# ejecutamos el número de epochs indicados
epochs = 1000
trace = 100
for i in range(1,epochs+1):
  sess.run(train_step, feed_dict={e: inputs, d: outputs})
  if i%trace == 0:
    print ('Epoch: %d' % i)
    # calculamos RMS y precisión y la escribimos por pantalla
    print('RMS:',sess.run(RMS, feed_dict={e: inputs, d: outputs}))
    print ('Accuracy:',sess.run(accuracy, feed_dict={e: inputs, d: outputs}))
    
sess.close()


Entrenamiento para datos circulo 

Epoch: 100
RMS: 0.45542932
Accuracy: 0.48
Epoch: 200
RMS: 0.45535135
Accuracy: 0.48
Epoch: 300
RMS: 0.41734877
Accuracy: 0.52
Epoch: 400
RMS: 0.3361843
Accuracy: 0.84
Epoch: 500
RMS: 0.27995613
Accuracy: 0.88
Epoch: 600
RMS: 0.24006271
Accuracy: 0.96
Epoch: 700
RMS: 0.2096458
Accuracy: 1.0
Epoch: 800
RMS: 0.18800426
Accuracy: 1.0
Epoch: 900
RMS: 0.17148672
Accuracy: 1.0
Epoch: 1000
RMS: 0.15687752
Accuracy: 1.0


### Entrenar con algunos de los problemas suministrados.
Para entrenar con un problema, ejecutar la carga de datos de ese problema y posteriormente ejecutar el entrenamiento 

In [23]:
# Cargar datos morosos 

ni = 9 # número de entradas
nh = 3 # número de neuronas en capa oculta
no = 1 # número de neuronas en capa de salida
capas = 2 #número de capas ocultas de la red
datos_cargados = "morosos"

# carga datos de entrenamiento
d = np.loadtxt("samples/morosos-ent.txt")
inputs = d[:,:ni]

outputs = d[:,ni:]

t = np.loadtxt("samples/morosos-tst.txt")
inputsT = t[:,:ni]
outputsT = t[:,ni:]

In [26]:
# Cargar data2 class

ni = 2 # número de entradas
nh = 7 # número de neuronas en capa oculta
no = 1 # número de neuronas en capa de salida
capas = 2 #número de capas ocultas de la red
datos_cargados = "data 3classes nonlinear"

# carga datos de entrenamiento
d = np.loadtxt("samples/data_3classes_nonlinear_2D.txt")
inputs = d[:,:ni]

outputs = d[:,ni:]

## Para este caso no hay conjunto de test 
t = np.loadtxt("samples/data_3classes_nonlinear_2D.txt")
inputsT = t[:,:ni]
outputsT = t[:,ni:]


In [36]:
# Cargar datos quinielas

ni = 60 # número de entradas
nh = 60 # número de neuronas en capa oculta
no = 3 # número de neuronas en capa de salida
capas = 25 #número de capas ocultas de la red
datos_cargados = "quinielas"

# carga datos de entrenamiento
d = np.loadtxt("samples/quinielas60-3-trn.txt")
inputs = d[:,:ni]
outputs = d[:,ni:]

t = np.loadtxt("samples/quinielas60-3-tst.txt")
inputsT = t[:,:ni]
outputsT = t[:,ni:]

In [32]:
# Cargar datos aprobados 

ni = 3 # número de entradas
nh = 7 # número de neuronas en capa oculta
no = 1 # número de neuronas en capa de salida
capas = 10 #número de capas ocultas de la red
datos_cargados = "aprobado"

# carga datos de entrenamiento
d = np.loadtxt("samples/aprobado-ent.txt")
inputs = d[:,:ni]
outputs = d[:,ni:]

t = np.loadtxt("samples/aprobado-tst.txt")
inputsT = t[:,:ni]
outputsT = t[:,ni:]



### Ampliar el perceptrón multicapa suministrado para meter más capas, usar activación relu y evaluar con un conjunto de test.



In [53]:
# Creación de la red
# Define un perceptrón multicapa con 2 capas usando RMS como medida del error y back proagation como algoritmo de entrenamiento

import tensorflow as tf

# un placeholder es un punto de comunicación de datos entre nuestra app y la librería tensorflow
# creamos un placeholder para indicar las entradas a la red de todos los ejemplos
e = tf.placeholder(tf.float32, [None, ni]) # None indica que la primera dimensión no es fija (el número de ejemplos)

# un nuevo placeholder para indicar las salidas deseadas
d = tf.placeholder(tf.float32, [None, no])


# las variables son matrices que residen fuera de python y no son modificadas directamente por nuestra app
W = []
b = []

W.append(tf.Variable(tf.glorot_uniform_initializer()((ni, nh),dtype=tf.float32)))
b.append(tf.Variable(tf.glorot_uniform_initializer()([nh], dtype=tf.float32)))
for _ in range (1,capas-1):
    W.append(tf.Variable(tf.glorot_uniform_initializer()((nh, nh), dtype=tf.float32)))
    b.append(tf.Variable(tf.glorot_uniform_initializer()([nh], dtype=tf.float32)))

W.append(tf.Variable(tf.glorot_uniform_initializer()((nh, no),  dtype=tf.float32)))
b.append(tf.Variable(tf.glorot_uniform_initializer()([no], dtype=tf.float32)))

# aquí definimos la propagación de la red
s = []
s.append(tf.nn.relu(tf.add(tf.matmul(e, W[0]), b[0])))
for i in range (capas - 1):
    s.append(tf.nn.relu(tf.add(tf.matmul(s[i], W[i+1]), b[i+1])))

print("\ndatos cargados:", datos_cargados,"\n")




datos cargados: quinielas 



In [54]:
# Entrenamiento

# definimos la función a minimizar (error cuadrático medio)
vRMS = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(d,s[capas-1])), reduction_indices=0))
RMS = tf.reduce_mean(vRMS)

# definimos el algoritmo de aprendizaje a utilizar (descenso del gradiente sobre la función de coste)
train_step = tf.train.AdamOptimizer().minimize(RMS)

# definimos qué es un acierto  
correct_prediction = tf.equal(tf.argmax(s[capas-1],1), tf.argmax(d,1))

# definimos la precisión como el porcentaje de aciertos
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# definimos la inicialización de las variables internas
init = tf.global_variables_initializer()

# iniciamos sesión con tensorflow y ejecutamos la inicialización
sess = tf.Session()
sess.run(init)

print("\nEntrenamiento para datos",datos_cargados, "\n")

# ejecutamos el número de epochs indicados
epochs = 1000
trace = 100
for i in range(1,epochs+1):
  sess.run(train_step, feed_dict={e: inputs, d: outputs})
  if i%trace == 0:
    print ('Epoch: %d' % i)
    # calculamos RMS y precisión y la escribimos por pantalla
    print('RMS:',sess.run(RMS, feed_dict={e: inputs, d: outputs}))
    print('RMS test:',sess.run(RMS, feed_dict={e: inputsT, d: outputsT}))
    print ('Accuracy:',sess.run(accuracy, feed_dict={e: inputs, d: outputs}))
    print ('Accuracy test:',sess.run(accuracy, feed_dict={e: inputsT, d: outputsT}))
    
sess.close()


Entrenamiento para datos quinielas 

Epoch: 100
RMS: 0.4430469
RMS test: 0.46161413
Accuracy: 0.6040404
Accuracy test: 0.53939396
Epoch: 200
RMS: 0.38536274
RMS test: 0.49511075
Accuracy: 0.7050505
Accuracy test: 0.5090909
Epoch: 300
RMS: 0.344424
RMS test: 0.5247931
Accuracy: 0.7164983
Accuracy test: 0.47878787
Epoch: 400
RMS: 0.31864813
RMS test: 0.5271068
Accuracy: 0.73670036
Accuracy test: 0.4848485
Epoch: 500
RMS: 0.3448087
RMS test: 0.55192435
Accuracy: 0.7414141
Accuracy test: 0.47272727
Epoch: 600
RMS: 0.29506326
RMS test: 0.5415749
Accuracy: 0.75151515
Accuracy test: 0.4848485
Epoch: 700
RMS: 0.43831134
RMS test: 0.5530704
Accuracy: 0.6936027
Accuracy test: 0.43636364
Epoch: 800
RMS: 0.28214943
RMS test: 0.54619104
Accuracy: 0.75824916
Accuracy test: 0.5030303
Epoch: 900
RMS: 0.2804253
RMS test: 0.5438335
Accuracy: 0.75959593
Accuracy test: 0.4969697
Epoch: 1000
RMS: 0.256274
RMS test: 0.5480623
Accuracy: 0.75959593
Accuracy test: 0.4909091


### Probar otros optimizadores

Se utilizaron los optimizadores:
- RMSPropOptimizer
- AdamOptimizer

###  Añadir dropout.

In [44]:
# Creación de la red DROPOUT

import tensorflow as tf

# un placeholder es un punto de comunicación de datos entre nuestra app y la librería tensorflow
# creamos un placeholder para indicar las entradas a la red de todos los ejemplos
e = tf.placeholder(tf.float32, [None, ni]) # None indica que la primera dimensión no es fija (el número de ejemplos)

# un nuevo placeholder para indicar las salidas deseadas
d = tf.placeholder(tf.float32, [None, no])

# las variables son matrices que residen fuera de python y no son modificadas directamente por nuestra app
W = []
b = []

W.append(tf.Variable(tf.glorot_uniform_initializer()((ni, nh),dtype=tf.float32)))
b.append(tf.Variable(tf.glorot_uniform_initializer()([nh], dtype=tf.float32)))
for _ in range (1,capas-1):
    W.append(tf.Variable(tf.glorot_uniform_initializer()((nh, nh), dtype=tf.float32)))
    b.append(tf.Variable(tf.glorot_uniform_initializer()([nh], dtype=tf.float32)))

W.append(tf.Variable(tf.glorot_uniform_initializer()((nh, no),  dtype=tf.float32)))
b.append(tf.Variable(tf.glorot_uniform_initializer()([no], dtype=tf.float32)))

# aquí definimos la propagación de la red con DROPOUT
s = []
s.append(tf.nn.relu(tf.add(tf.matmul(e, W[0]), b[0])))

y = tf.placeholder("float", [None, 3])
keep_prob = tf.placeholder(tf.float32)  
drop_out = tf.nn.dropout(s[0], keep_prob) 

s.append(tf.matmul(s[0], W[1]) + b[1]) #DROPOUT primera capa oculta

for i in range (1, capas - 1):
    s.append(tf.nn.relu(tf.add(tf.matmul(s[i], W[i+1]), b[i+1])))# Entrenamiento CON DROPOUT 


In [45]:
#Entramiento de la red con DROPOUT

vRMS = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(d,s[capas-1])), reduction_indices=0))
RMS = tf.reduce_mean(vRMS)

# definimos el algoritmo de aprendizaje a utilizar (descenso del gradiente sobre la función de coste)
train_step = tf.train.AdamOptimizer().minimize(RMS)

# definimos qué es un acierto  
correct_prediction = tf.equal(tf.argmax(s[capas-1],1), tf.argmax(d,1))

# definimos la precisión como el porcentaje de aciertos
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# definimos la inicialización de las variables internas
init = tf.global_variables_initializer()

# iniciamos sesión con tensorflow y ejecutamos la inicialización
sess = tf.Session()
sess.run(init)

# ejecutamos el número de epochs indicados
epochs = 1000
trace = 100
for i in range(1,epochs+1):
  sess.run(train_step, feed_dict={e: inputs, d: outputs})
  if i%trace == 0:
    print ('Epoch: %d' % i)
    # calculamos RMS y precisión y la escribimos por pantalla
    print('RMS:',sess.run(RMS, feed_dict={e: inputs, d: outputs}))
    print('RMS test:',sess.run(RMS, feed_dict={e: inputsT, d: outputsT}))
    print ('Accuracy:',sess.run(accuracy, feed_dict={e: inputs, d: outputs}))
    print ('Accuracy test:',sess.run(accuracy, feed_dict={e: inputsT, d: outputsT}))
    
sess.close()

Epoch: 100
RMS: 0.5277136
RMS test: 0.5379788
Accuracy: 0.26599327
Accuracy test: 0.27272728
Epoch: 200
RMS: 0.4824904
RMS test: 0.5680337
Accuracy: 0.41144782
Accuracy test: 0.33333334
Epoch: 300
RMS: 0.47418436
RMS test: 0.5725417
Accuracy: 0.71784514
Accuracy test: 0.5030303
Epoch: 400
RMS: 0.48620343
RMS test: 0.57915884
Accuracy: 0.7037037
Accuracy test: 0.4969697
Epoch: 500
RMS: 0.47003445
RMS test: 0.5789849
Accuracy: 0.72390574
Accuracy test: 0.4969697
Epoch: 600
RMS: 0.47001544
RMS test: 0.57948685
Accuracy: 0.7225589
Accuracy test: 0.4969697
Epoch: 700
RMS: 0.4700007
RMS test: 0.5802353
Accuracy: 0.72390574
Accuracy test: 0.4848485
Epoch: 800
RMS: 0.46996245
RMS test: 0.58101946
Accuracy: 0.72390574
Accuracy test: 0.47878787
Epoch: 900
RMS: 0.46980783
RMS test: 0.58117914
Accuracy: 0.7232323
Accuracy test: 0.4848485
Epoch: 1000
RMS: 0.4695574
RMS test: 0.5815864
Accuracy: 0.7245791
Accuracy test: 0.4848485


### Explicación DROPOUT (datos quinielas)

Se cargan los datos de quinielas, es el mas complejo de los que hemos decidido cargar en esta práctica, y al realizar el entrenamiento con dropout y sin dropout hemos podido observar que con dropout se generaliza un poco mas la red y, por lo tanto, en la precisión (accuracy) de los datos en test es algo mejor que en el entramiento.
Sin dropout el entramiento nos ha llegado a salir una precisión muy buena (en torno al 0.9) y, sin embargo, en el test dan resultados bastante malos. Con el dropout no conseguimos tanta precisión (lo máximo alrededor 0.85), pero en el test si conseguimos mejores resultados debidos a dicha generalización.

Esto se dará en los demás ejemplos pero este es mas visible pues no hemos llegado a lograr el 1.0 de precisión.
 